# Preprocess data for viz

## Data for map 

In [24]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

plr = gpd.read_file('lor_planungsraeume.geojson', encoding='utf-8')

# old data: stgbxIIlor.csv
df = pd.read_csv('2018_new.csv', sep=";", dtype={'Kennung':str}, na_values=['.', 'x'], encoding='utf-8')
df.set_index('Kennung', inplace=True)

data = df.join(plr.set_index('spatial_name'), rsuffix='_r')

data.rename(columns={
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\ninsgesamt)': '18-64 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'ab 65 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nDeutsche)': '18-64 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nDeutsche)': 'ab 65 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nAusländer)': '18-64 Jahre Ausländer',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nAusländer)': 'ab 65 Jahre Ausländer'
}, inplace=True)

gpd.GeoDataFrame(data).to_file('../data/xii.geojson', driver='GeoJSON', encoding='utf-8')

## Merge all years for whole Berlin

In [ ]:
full_df = pd.DataFrame()

for i in range (2006, 2019):
    df = pd.read_excel(str(i) + '.xls', sheet_name='Tab E1', header=2, na_values=['x', '.'])
    df['jahr'] = i
    full_df = full_df.append(df[df.iloc[:,1] == 'Berlin'], sort=False)

full_df.set_index('jahr', inplace = True)
full_df = full_df.round(2)

full_df.rename(columns={
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\ninsgesamt)': '18-64 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'ab 65 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nDeutsche)': '18-64 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nDeutsche)': 'ab 65 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nAusländer)': '18-64 Jahre Ausländer',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nAusländer)': 'ab 65 Jahre Ausländer'
}, inplace=True)

full_df[['18-64 Jahre insgesamt',
       'ab 65 Jahre insgesamt', '18-64 Jahre Deutsche', 'ab 65 Jahre Deutsche',
       '18-64 Jahre Ausländer', 'ab 65 Jahre Ausländer']].to_csv('timeline.csv', encoding='utf-8')

full_df = full_df.round(2)
full_df.to_csv('timeline.csv', encoding='utf-8')

## Merge all years on LOR Level

In [25]:
df = pd.DataFrame()

# ab 65 Jahre
df = pd.DataFrame()

for i in range (2006, 2019):
    df_temp = pd.read_excel(str(i) + '.xls', sheet_name='Tab E1', header=2, na_values=['x', '.'])
    df_temp.rename(columns={' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'y' + str(i), 
                            'Planungsraum':'Kennung', 'Unnamed: 1':'Name'}, inplace=True)

    df_temp = df_temp[df_temp.Name.notnull() & (df_temp.Name != 'Name')]
    df_temp = df_temp[['Kennung', 'Name', 'y' + str(i)]]
    
    if df.size == 0:
        df = df_temp
    else:
        df = df.merge(df_temp, how="right")

df = df.round(2)

df = df.fillna('null')

df.to_csv('timelapse.csv', encoding='utf-8')